In [1]:
import requests
from bs4 import BeautifulSoup
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

In [2]:
def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

In [3]:
def split_text(text, chunk_size=500):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


In [4]:
llm = OllamaLLM(model="llama3.2", server_url="http://localhost:11434") 

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/bigdata/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
vectorstore = Chroma(persist_directory="./vectorstore", embedding_function=embedding_model)

In [7]:
url = "https://en.wikipedia.org/wiki/2024_G20_Rio_de_Janeiro_summit"
page_content = extract_text_from_url(url)

In [8]:
chunks = split_text(page_content)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 35


In [9]:
documents = [Document(page_content=chunk) for chunk in chunks]
vectorstore.add_documents(documents)
print("Documents added to the vector store.")

Documents added to the vector store.


In [10]:
prompt = ChatPromptTemplate.from_template(
    template="Use the context below to answer the user's question:\n\n{context}\n\nQuestion: {question}\nAnswer:"
)

In [11]:
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [12]:
query = "What is the main topic of the webpage?"
response = qa_chain.run(query)
print("Response:", response)

/tmp/ipykernel_21151/1094954513.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Response: The main topic of the webpage appears to be the 2024 G20 Rio de Janeiro summit, specifically highlighting its attendees, agenda themes, and participating leaders.
